In [1]:
from PIL import Image
from torchvision import transforms

import torch
import matplotlib.pyplot as plt

In [2]:
from os import listdir
from os.path import isfile, join

prefix =  "./wow_airlines/"

onlyfiles = [prefix+f for f in listdir("./wow_airlines") if isfile(join("./wow_airlines", f))]

In [3]:
print(onlyfiles)

['./wow_airlines/images - 2020-05-09T093951.270.jpg', './wow_airlines/images - 2020-05-09T093951.120.jpg', './wow_airlines/images - 2020-05-09T094008.229.jpg', './wow_airlines/images (88).jpg', './wow_airlines/images - 2020-05-09T094030.895.jpg', './wow_airlines/images - 2020-05-09T093951.970.jpg', './wow_airlines/images (11).jpg', './wow_airlines/images - 2020-05-09T093956.065.jpg', './wow_airlines/images - 2020-05-09T093959.188.jpg', './wow_airlines/images - 2020-05-09T093954.438.jpg', './wow_airlines/images - 2020-05-09T093957.337.jpg', './wow_airlines/images (61).jpg', './wow_airlines/images - 2020-05-09T093956.905.jpg', './wow_airlines/images - 2020-05-09T094011.264.jpg', './wow_airlines/images (58).jpg', './wow_airlines/images (90).jpg', './wow_airlines/2Q__ (3).jpg', './wow_airlines/images (43).jpg', './wow_airlines/images - 2020-05-09T094030.213.jpg', './wow_airlines/images - 2020-05-09T093951.684.jpg', './wow_airlines/images - 2020-05-09T094013.927.jpg', './wow_airlines/images

In [4]:
from PIL import Image
from torchvision import transforms
import numpy as np

import torch
import matplotlib.pyplot as plt

import random

In [5]:
random.seed(0)
np.random.seed(0)

In [6]:
img_list = []

In [7]:
for i, file_path in enumerate(onlyfiles):
    img_list.append(Image.open(file_path))

In [8]:
def trans_pil2CIFAR(img):
    trans_resize = transforms.Resize(size=(32, 32))
    resized_img = trans_resize(img)
    return resized_img

def trans_randRotate(img):
    trans_rotate = transforms.RandomAffine(degrees=(-180, 180))
    rotated_img = trans_rotate(img)
    return rotated_img

def trans_pil2tensor(pil_img):
    trans_pil2t = transforms.ToTensor()
    transformed_numpy = trans_pil2t(pil_img).numpy()  # convert to HWC
    return (transformed_numpy * 255).astype(np.uint8)

In [9]:
transformed_img_list_raw = []
transformed_img_list_dummy_augment = []

In [10]:
indices_southwest = np.arange(len(img_list))

for raw_idx in indices_southwest:
    img_raw = img_list[raw_idx]
    img_cifar_size_raw = trans_pil2CIFAR(img_raw)
    transformed_img_list_raw.append(img_cifar_size_raw)

In [11]:
fraction=3
cand_angles = [90 * i for i in range(1, fraction+1)]

for raw_idx in indices_southwest:
    img_dummy_augment = img_list[raw_idx]
    img_cifar_size_dummy_augment = trans_pil2CIFAR(img_dummy_augment)
    transformed_img_list_dummy_augment.append(img_cifar_size_dummy_augment)
    for cad_ang in cand_angles:
        img_rotated_dummy_augment = transforms.functional.rotate(img_cifar_size_dummy_augment, cad_ang)
        transformed_img_list_dummy_augment.append(img_rotated_dummy_augment)

In [12]:
for trans_idx_dummy_augment, trans_imgs_dummy_augment in enumerate(transformed_img_list_dummy_augment):
    trans_imgs_dummy_augment.save("./wow_airlines_preprocessed/dummy_augment/processed_img{}.jpg".format(trans_idx_dummy_augment))

In [13]:
transformed_tensor_dummy_augment = [trans_pil2tensor(til) for til in transformed_img_list_dummy_augment]

In [14]:
concat_tt_dummy_augment = np.stack(transformed_tensor_dummy_augment, axis=0).transpose((0, 2, 3, 1))  # convert to HWC
print(concat_tt_dummy_augment.shape)

import pickle
with open('wow_images_new_dummy_augment.pkl', 'wb') as f:
    pickle.dump(concat_tt_dummy_augment, f)

(736, 32, 32, 3)


In [15]:
for trans_idx_raw, trans_imgs_raw in enumerate(transformed_img_list_raw):
    trans_imgs_raw.save("./wow_airlines_preprocessed/raw/processed_img{}.jpg".format(trans_idx_raw))

In [16]:
transformed_tensor_raw = [trans_pil2tensor(til) for til in transformed_img_list_raw]

In [17]:
concat_tt_raw = np.stack(transformed_tensor_raw, axis=0).transpose((0, 2, 3, 1))  # convert to HWC
print(concat_tt_raw.shape)

import pickle
with open('wow_images_new_raw.pkl', 'wb') as f:
    pickle.dump(concat_tt_raw, f)

(184, 32, 32, 3)


# Let's split the images into training and test set

In [18]:
indices_southwest = np.arange(len(img_list))
permuted_indices = np.random.permutation(indices_southwest)

In [19]:
print(int(len(permuted_indices)*0.2))
print(int(len(permuted_indices)*0.8))
print(int(len(permuted_indices)*0.2)+int(len(permuted_indices)*0.8))
print(len(permuted_indices))
num_images_train = int(len(permuted_indices)*0.8)
num_images_test = int(len(permuted_indices)*0.2)

36
147
183
184


In [20]:
permuted_indices_train = permuted_indices[0:num_images_train]
permuted_indices_test = permuted_indices[num_images_train:]

In [21]:
print(len(permuted_indices_train))
print(len(permuted_indices_test))

147
37


In [22]:
print(permuted_indices_train)
print()
print(permuted_indices_test)

[139 106   7 107  60  97  61 166  33 170 163  71   5 113 151 146  18  66
 150  74 167 160  56 174 156   4  54 131 118 123 112  63 111  26  37 134
  95   8  45  86  44 116  55  24  30  98 130  83  90  19 183 143 149  16
  51  80 126  40 124  22 136 169 137 109  93 121  27 168 179 125 180  92
  62   2  59 108 101  43  10  89  73 177 164 181 110  96 145 176  50 122
  94  64 129  41  69  49  48  85  13 138  23 171 141  20  15  78 104  52
 100  76   3 182 154   6  68  75  84 135  12 153 157  14   0  91 158  46
  11 119 102  35  57  65   1 120 161  42 105 132 159  17  38 133  53 144
 128  34  28]

[114  31 152 127 162  32 142 155 147  29  99  82  79 115 148 178  72  77
  25 165  81 173 175  39  58 140  88  70  87  36  21   9 103  67 117  47
 172]


In [23]:
transformed_img_list_train = []
transformed_img_list_test = []

In [25]:
fraction=3
cand_angles = [90 * i for i in range(1, fraction+1)]

# we will need to split the images into training and test sets
for train_idx in permuted_indices_train:
    img_train = img_list[train_idx]
    img_cifdar_size_train = trans_pil2CIFAR(img_train)
    transformed_img_list_train.append(img_cifdar_size_train)
    for cad_ang in cand_angles:
        img_rotated_train = transforms.functional.rotate(img_cifdar_size_train, cad_ang)
        transformed_img_list_train.append(img_rotated_train)

# we will need to split the images into training and test sets
for test_idx in permuted_indices_test:
    img_test = img_list[test_idx]
    img_cifdar_size_test = trans_pil2CIFAR(img_test)
    transformed_img_list_test.append(img_cifdar_size_test)
    for cad_ang in cand_angles:
        img_rotated_test = transforms.functional.rotate(img_cifdar_size_test, cad_ang)
        transformed_img_list_test.append(img_rotated_test)

In [26]:
print(len(transformed_img_list_train))
print(len(transformed_img_list_test))

588
148


In [27]:
for trans_idx_train, trans_imgs_train in enumerate(transformed_img_list_train):
    trans_imgs_train.save("./wow_airlines_preprocessed/train/processed_img{}.jpg".format(trans_idx_train)) 
    
for trans_idx_test, trans_imgs_test in enumerate(transformed_img_list_test):
    trans_imgs_test.save("./wow_airlines_preprocessed/test/processed_img{}.jpg".format(trans_idx_test)) 

In [28]:
transformed_tensor_train = [trans_pil2tensor(til) for til in transformed_img_list_train]
transformed_tensor_test = [trans_pil2tensor(til) for til in transformed_img_list_test]

In [29]:
concat_tt_train = np.stack(transformed_tensor_train, axis=0).transpose((0, 2, 3, 1))  # convert to HWC
print(concat_tt_train.shape)

concat_tt_test = np.stack(transformed_tensor_test, axis=0).transpose((0, 2, 3, 1))  # convert to HWC
print(concat_tt_test.shape)

import pickle
with open('wow_images_new_train.pkl', 'wb') as f:
    pickle.dump(concat_tt_train, f)
    
with open('wow_images_new_test.pkl', 'wb') as f:
    pickle.dump(concat_tt_test, f)

(588, 32, 32, 3)
(148, 32, 32, 3)


## Combine the whole WOW dataset

In [31]:
transformed_img_list_whole = []

In [32]:
fraction=3
cand_angles = [90 * i for i in range(1, fraction+1)]

# we will need to split the images into training and test sets
for whole_idx in permuted_indices:
    img_whole = img_list[whole_idx]
    img_cifdar_size_whole = trans_pil2CIFAR(img_whole)
    transformed_img_list_whole.append(img_cifdar_size_whole)
    for cad_ang in cand_angles:
        img_rotated_whole = transforms.functional.rotate(img_cifdar_size_whole, cad_ang)
        transformed_img_list_whole.append(img_rotated_whole)

In [33]:
print(len(transformed_img_list_whole))

736


In [34]:
transformed_tensor_train = [trans_pil2tensor(til) for til in transformed_img_list_whole]

In [35]:
concat_tt_whole = np.stack(transformed_tensor_train, axis=0).transpose((0, 2, 3, 1))  # convert to HWC
print(concat_tt_whole.shape)

(736, 32, 32, 3)


In [36]:
import pickle
with open('wow_images_new_whole.pkl', 'wb') as f:
    pickle.dump(concat_tt_whole, f)